In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
CW_DATASET_PATH = "CW_dataset"

def load_data(dataset_path=CW_DATASET_PATH):
    x_train_all_path = os.path.join(dataset_path, "x_train_all.csv")
    y_train_all_path = os.path.join(dataset_path, "y_train_all.csv")
    x_test_all_path = os.path.join(dataset_path, "x_test_all.csv")
    y_test_all_path = os.path.join(dataset_path, "y_test_all.csv")

    x_train_all = pd.read_csv(x_train_all_path)
    y_train_all = pd.read_csv(y_train_all_path)
    x_test_all = pd.read_csv(x_test_all_path)
    y_test_all = pd.read_csv(y_test_all_path)

    return x_train_all, y_train_all,x_test_all,y_test_all

In [3]:
x_train, y_train,x_test, y_test = load_data()

In [5]:
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.metrics import accuracy_score, classification_report

clf = DecisionTreeClassifier(random_state=42)
clf.fit(x_train, y_train) 

y_pred = clf.predict(x_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on the test set:", test_accuracy)

classification_rep = classification_report(y_test, y_pred)
print(classification_rep)


Accuracy on the test set: 0.6854368932038835
              precision    recall  f1-score   support

           0       0.27      0.32      0.29        60
           1       0.72      0.78      0.75       720
           2       0.76      0.74      0.75       750
           3       0.67      0.69      0.68       450
           4       0.73      0.72      0.72       660
           5       0.43      0.48      0.46        60
           6       0.46      0.40      0.43        90
           7       0.27      0.30      0.29        60
           8       0.57      0.43      0.49       150
           9       0.65      0.51      0.57        90

    accuracy                           0.69      3090
   macro avg       0.55      0.54      0.54      3090
weighted avg       0.69      0.69      0.68      3090

